<a href="https://colab.research.google.com/github/rafaelsguerra/information_retrieval/blob/master/Lab%2006%20-%20Vectorial%20Model/Modelo_Vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import collections
import pandas as pd
import numpy as np
import re
import nltk
import heapq
import math
import itertools

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/rafaelsguerra/information_retrieval/master/data/results.csv')
data_title = data['title'].tolist()
data_subtitle = data['subtitle'].tolist()
data_text = data['text'].tolist()

In [0]:
def parse(document):  
  tokenizer = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+') 
  tokens = []

  tokens.extend(tokenizer.tokenize(document.lower()))
    
  stop_words = stopwords.words('portuguese')

  filtered_tokens = []

  for token in tokens:
    if token not in stop_words  and len(token) >= 2: # and len(token) > 2
      filtered_tokens.append(token)
  
  return filtered_tokens

## Questões 1 e 2
Reconstuir o índice invertido e incluir o IDF

In [0]:
# Retorna o índice invertido
# Cada palavra tem uma lista invertida associada, no formato (score, documento)
# O último elemento da lista invertida será o IDF associado à palavra

def build_index(data): 
  inverted_list = {}
  n = 0
  number_of_docs = len(data_text)
  
  for document in data:
    n = n + 1
    tokens = parse(document)
    
    tokens_no_duplicates = collections.Counter(tokens)
    
    for token in tokens_no_duplicates.keys():
      if token not in inverted_list:
        inverted_list[token] = []
      inverted_list[token].append((n, tokens_no_duplicates[token]))
    
  for token in inverted_list.keys():                                       # Neste laço é adicionado à lista invertida de cada palavra do documento seu respectivo idf
    idf = math.log10((number_of_docs + 1) / len(inverted_list[token]))
    inverted_list[token].append(idf)
    
    
  return inverted_list

In [0]:
inverted_index = build_index(data_text)

In [0]:
#Questão 3

## Questão 3
Implementação das versões do modelo vetorial: representação binária, TF, TF-IDF e BM25

In [0]:
def binary_rep(query, document_index, data):
  score = 0
  
  query = parse(query)
  
  for word in set(query):
    if word in data[document_index] :
      score += 1
  
  return score

In [0]:
def tf(query, document_index, data):
  score = 0
  
  query = parse(query)  
  document = parse(data[document_index])
  
  counter_query = collections.Counter(query)
  counter_doc = collections.Counter(document)
  
  for word in set(query):
    if word in data[document_index]:
      score += counter_query[word] * counter_doc[word]
      
  return score  

In [0]:
def tf_idf(query, document_index, data, inverted_index):
  score = 0
  
  query = parse(query)  
  document = parse(data[document_index])
  
  counter_query = collections.Counter(query)
  counter_doc = collections.Counter(document)
  
  for word in set(query):
    if word in data[document_index]:
      score += counter_query[word] * (counter_doc[word] * inverted_index[word][-1])
      
  return score  

In [0]:
def bm_25(query, document_index, data, inverted_index, k):
  score = 0
  
  query = parse(query)
  document = parse(data[document_index])
  
  for word in list(set(query)):
    count_word_query = query.count(word)
    count_word_doc = document.count(word)
    y = ((k + 1) * count_word_doc) / (count_word_doc + k)
    document_frequency = len(inverted_index[word]) - 1
    
    score += count_word_query * y * math.log10((len(data) + 1) / document_frequency)
    
  return score  

Aqui foi determinado um valor para k de modo a obter uma suavização dos resultados para scores muito grandes. Para isso, foi escolhida uma consulta e um documento aleatórios. Vejamos como o resultado da consulta se altera, dado determinado k. Observe como o ritmo de crescimento do resultado diminui, à medida que o valor de k cresce: a diferença dos resultados entre k = 1 e k = 10 é de aproximadamente 3.8, enquanto que a diferença entre k = 10 e k = 20 é aproximadamente 0.78.

In [11]:
k = 1
k_values = []
results = []

for k in [x * 0.1 for x in range(0,201,10)]:
  res = bm_25("golpe de estado",165, data_text, inverted_index, k)
  results.append(res)
  k_values.append(k)
  
df = pd.DataFrame({'k':k_values, 'resultado bm25':results})
df

,k,resultado bm25
0,0.0,1.561363
1,1.0,2.483919
2,2.0,3.133682
3,3.0,3.624066
4,4.0,4.009628
5,5.0,4.321584
6,6.0,4.579540
7,7.0,4.796575
8,8.0,4.981803
9,9.0,5.141788


Neste lab não foi feita normalização do tamanho dos documentos, visto que eles não se alteram muito. Portanto, ao desconsiderar essa normalização, um valor muito comum para K, de acordo com experimentos feitos pela TREC, é 1.2, que será usado de agora em diante.

## Questão 4
Executar as implementações em 3 queries diferentes, retornando os top 5 documentos mais similares a cada query

In [0]:
from IPython.display import display, HTML
pd.options.display.max_colwidth = 500

In [0]:
def generate_table(docs, titles, subtitles):
  df = {'title': [], 'subtitle': []}
  for document in docs:
    df['title'].append(titles[document])
    df['subtitle'].append(subtitles[document])

  df = pd.DataFrame.from_dict(df)
  return df

In [14]:
queries = ['golpe de estado', 'campeonato brasileiro', 'crise na venezuela']

for query in queries:
  
  scores_br = {}
  scores_tf = {}
  scores_tf_idf = {}
  scores_bm_25 = {}

  for document in list(range(len(data_text))):
  
    scores_br[document] = binary_rep(query, document, data_text)
    scores_tf[document] = tf(query, document, data_text)
    scores_tf_idf[document] = tf_idf(query, document, data_text, inverted_index)
    scores_bm_25[document] = bm_25(query, document, data_text, inverted_index, 1.2)

  top5_scores_br_docs = heapq.nlargest(5, scores_br, key=scores_br.get)
  top5_scores_tf_docs = heapq.nlargest(5, scores_tf, key=scores_tf.get)
  top5_scores_tf_idf_docs = heapq.nlargest(5, scores_tf_idf, key=scores_tf_idf.get)
  top5_scores_bm_25_docs = heapq.nlargest(5, scores_bm_25, key=scores_bm_25.get)
  
  print('Utilizando a query \"' + query + '\":')
  print("Método da representação binária")
  display(generate_table(top5_scores_br_docs, data_title, data_subtitle))
  print('\n')

  print("Método tf")
  display(generate_table(top5_scores_tf_docs, data_title, data_subtitle))
  print('\n')

  print("Método tf-idf")
  display(generate_table(top5_scores_tf_idf_docs, data_title, data_subtitle))
  print('\n')

  print("Método bm-25")
  display(generate_table(top5_scores_bm_25_docs, data_title, data_subtitle))
  print('\n')

Utilizando a query "golpe de estado":
Método da representação binária


,title,subtitle
0,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Historiador diz que as Forças Armadas nunca reconheceram os aspectos mais negativos do regime militar e que não havia ameaça imediata de implantação de um regime comunista
1,Milícia o exército popular a serviço de Maduro,O poder do líder bolivariano na Venezuela se baseia no exército e nas milícias um corpo de um milhão de pessoas treinadas com vassouras e rifles que prometem dar suas vidas pela Revolução
2,“Não me arrependo de nada”,Procurador do caso Herzog que acobertou torturadores e pediu pena de morte para militantes Durval Ayrton Moura de Araújo de 99 anos conversa com o EL PAÍS
3,Quem pagou pelo vídeo revisionista da ditadura distribuído pelo Governo Bolsonaro?,“É preciso saber se foi pago com recursos públicos” diz Gil Castello Branco do Contas Abertas. Ao 'Globo' ator disse ter sido pago. Mourão disse que ideia foi de Bolsonaro
4,Ministro promete mudar livros didáticos por “visão mais ampla” da ditadura,Em entrevista ao 'Valor' Ricardo Vélez da Educação diz que não ocorreu um golpe em 1964




Método tf


,title,subtitle
0,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Historiador diz que as Forças Armadas nunca reconheceram os aspectos mais negativos do regime militar e que não havia ameaça imediata de implantação de um regime comunista
1,“Lógica de usar torturadores da ditadura no crime foi usada nas milícias”,Aloy Jupiara coautor de 'Os porões da contravenção' fala sobre o legado de profissionalização do crime deixado pelos anos de chumbo que perdura até hoje
2,As três espanholas do Estado Islâmico: “Só queremos ir embora”,EL PAÍS localiza em meio a milhares de famílias jihadistas num campo de acolhida três mulheres que viajaram com seus maridos para a Síria em 2014 e sobreviveram ao desmoronamento do califado
3,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Marcos Napolitano professor da USP diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo pois “tem um ponto de partida ideológico com objetivo de ocultar o passado”
4,Bolsonaro manda festejar o crime,Ao determinar a comemoração do golpe militar de 1964 o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça




Método tf-idf


...

,title,subtitle
0,Grupos que sustentam Maduro serão principal desafio a qualquer futuro governo na Venezuela,É erro grave presumir que queda de Maduro e promoção de eleições livres seriam garantia para resolver problemas do país vizinho
1,Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”,Em entrevista ao EL PAÍS o ideólogo da extrema direita diz que o vice-primeiro-ministro italiano e o premiê húngaro Viktor Orbán são os políticos mais importantes da Europa atualmente
2,O incerto futuro dos militares que desertaram na Venezuela,Mais de mil venezuelanos que desertaram há um mês aguardam em casas hotéis e albergues sem esconder seu mal-estar quanto às condições em que vivem
3,Adam Tooze: “Bolsonaro é terrível mas é a Itália que poderia quebrar a economia mundial”,Economista britânico Adam Tooze analisa os riscos da ascensão do populismo nas Américas e na Europa
4,Milícia o exército popular a serviço de Maduro,O poder do líder bolivariano na Venezuela se baseia no exército e nas milícias um corpo de um milhão de pessoas treinadas com vassouras e rifles que prometem dar suas vidas pela Revolução




Método tf-idf


,title,subtitle
0,Grupos que sustentam Maduro serão principal desafio a qualquer futuro governo na Venezuela,É erro grave presumir que queda de Maduro e promoção de eleições livres seriam garantia para resolver problemas do país vizinho
1,O incerto futuro dos militares que desertaram na Venezuela,Mais de mil venezuelanos que desertaram há um mês aguardam em casas hotéis e albergues sem esconder seu mal-estar quanto às condições em que vivem
2,Milícia o exército popular a serviço de Maduro,O poder do líder bolivariano na Venezuela se baseia no exército e nas milícias um corpo de um milhão de pessoas treinadas com vassouras e rifles que prometem dar suas vidas pela Revolução
3,Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”,Em entrevista ao EL PAÍS o ideólogo da extrema direita diz que o vice-primeiro-ministro italiano e o premiê húngaro Viktor Orbán são os políticos mais importantes da Europa atualmente
4,Adam Tooze: “Bolsonaro é terrível mas é a Itália que poderia quebrar a economia mundial”,Economista britânico Adam Tooze analisa os riscos da ascensão do populismo nas Américas e na Europa




Método bm-25


,title,subtitle
0,Grupos que sustentam Maduro serão principal desafio a qualquer futuro governo na Venezuela,É erro grave presumir que queda de Maduro e promoção de eleições livres seriam garantia para resolver problemas do país vizinho
1,Boeing enfrenta crise mais difícil de sua história,Empresa teme perda de confiança depois da morte de 345 pessoas em dois acidentes aéreos em cinco meses
2,Trump: “A Rússia tem que ir embora da Venezuela”,Presidente dos EUA recebe a mulher de Guaidó e sobe o tom de sua ameaça a Maduro:
3,A ética da solidariedade feminista,Se é preciso ouvir antes de falar é também nosso dever falar com responsabilidade sobre as injustiças que assolam as mulheres e meninas
4,Cruz Vermelha afirma que distribuirá ajuda humanitária na Venezuela em 15 dias,"Operação que pretende ajudar 650.000 pessoas será ""muito semelhante à da Síria"" afirma o presidente da organização internacional"


In [0]:
#Questão 5

## Questão 5
Comparar os resultados das versões implementadas e apresentar um relatório de overlap par a par entre os resultados de cada versão do modelo vetorial, utilizando o [índice de Jaccard](https://en.wikipedia.org/wiki/Jaccard_index)

Para a consulta "golpe de estado", as melhores versões do modelo vetorial foram o TF-IDF e o BM25, pois seus resultados se assemelharam mais com a consulta, mas o TF-IDF neste caso possui uma leve vantagem, pois seu top 5 é mais relevante.

Para a consulta "campeonato brasileiro", novamente as versões TF-IDF e BM25 se saíram melhor. É importante observar que, apenas o top 5 da versão BM25 apresentou somente notícias com assunto relacionado com a consulta, enquanto os outros modelos apresentado uma ou mais notícias com assuntos diferentes. Talvez isso se dê ao fato de que há poucos documentos que apresentam este assunto na coleção.

Por ultimo, para a consulta "crise na venezuela", todas as versões mostraram resultados, mas ao meu ver, nenhum deles foi realmente satisfatório. Dentre as versões, o destaque é para o TF-IDF que, mesmo com seu top 5 apresentando notícias irrelevantes, suas 3 primeiras posições se assemelham bastante com a consulta.

In [16]:
for query in queries:
  
  scores_br = {}
  scores_tf = {}
  scores_tf_idf = {}
  scores_bm_25 = {}

  for document in list(range(len(data_text))):
  
    scores_br[document] = binary_rep(query, document, data_text)
    scores_tf[document] = tf(query, document, data_text)
    scores_tf_idf[document] = tf_idf(query, document, data_text, inverted_index)
    scores_bm_25[document] = bm_25(query, document, data_text, inverted_index, 1.2)

  top5_scores_br_docs = heapq.nlargest(5, scores_br, key=scores_br.get)
  top5_scores_tf_docs = heapq.nlargest(5, scores_tf, key=scores_tf.get)
  top5_scores_tf_idf_docs = heapq.nlargest(5, scores_tf_idf, key=scores_tf_idf.get)
  top5_scores_bm_25_docs = heapq.nlargest(5, scores_bm_25, key=scores_bm_25.get)

  df = {'binary_rep': top5_scores_br_docs, 'tf': top5_scores_tf_docs, 'tf_idf': top5_scores_tf_idf_docs, 'bm_25': top5_scores_bm_25_docs}
  df = pd.DataFrame.from_dict(df)
  
  funs = [binary_rep, tf, tf_idf, bm_25]
  fun_names = [(x.__name__,y.__name__) for x,y in [tup for tup in list(itertools.combinations(funs,2))]]

  print(f'Abaixo estão os resultados da consulta "{query}":')
  for fun1,fun2 in fun_names:
    inter = len(set(df[fun1]) & set(df[fun2]))
    union = len(set(df[fun1])) + len(set(df[fun2])) - inter
    jaccard = inter/union
    print(f'O resultado entre {fun1} e {fun2} foi {jaccard:.3}.')
  print('\n')

Abaixo estão os resultados da consulta "golpe de estado":
O resultado entre binary_rep e tf foi 0.111.
O resultado entre binary_rep e tf_idf foi 0.111.
O resultado entre binary_rep e bm_25 foi 0.111.
O resultado entre tf e tf_idf foi 0.429.
O resultado entre tf e bm_25 foi 0.429.
O resultado entre tf_idf e bm_25 foi 0.667.


Abaixo estão os resultados da consulta "campeonato brasileiro":
O resultado entre binary_rep e tf foi 0.111.
O resultado entre binary_rep e tf_idf foi 0.111.
O resultado entre binary_rep e bm_25 foi 0.111.
O resultado entre tf e tf_idf foi 0.667.
O resultado entre tf e bm_25 foi 0.25.
O resultado entre tf_idf e bm_25 foi 0.429.


Abaixo estão os resultados da consulta "crise na venezuela":
O resultado entre binary_rep e tf foi 0.111.
O resultado entre binary_rep e tf_idf foi 0.111.
O resultado entre binary_rep e bm_25 foi 0.429.
O resultado entre tf e tf_idf foi 1.0.
O resultado entre tf e bm_25 foi 0.111.
O resultado entre tf_idf e bm_25 foi 0.111.


